In [44]:
#Importing libraries
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install "scikit-learn==1.5.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 151.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 173.5 MB/s eta 0:00:0000:01


In [11]:
#Setting OS

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#imports
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

#Getting Data
filepath='https://cocl.us/concrete_data'
concrete_data = pd.read_csv(filepath)


In [12]:
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [13]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [14]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [30]:
concrete_data_columns = concrete_data.columns

In [36]:
predictors = concrete_data[['Cement','Blast Furnace Slag','Fly Ash','Water','Superplasticizer','Coarse Aggregate','Fine Aggregate']]
target = concrete_data['Strength'] # Strength column
predictors.head()
#target.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [37]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [38]:
n_cols = predictors_norm.shape[1] # number of predictors
print(n_cols)

7


In [39]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# build the model
model = regression_model()
MeanSquareError = []
for x in range(50):
    print("Step : ",x," started...")
    predictors_norm_train, predictors_norm_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.30, random_state=42)
    model.fit(predictors_norm_train, target_train, validation_split=0.3, epochs=50, verbose=0)
    target_prediction = model.predict(predictors_norm_test)
    MeanSquareError.append(mean_squared_error(target_test, target_prediction))
    print("Step : ",x," done.")
    


Step :  0  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Step :  0  done.
Step :  1  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Step :  1  done.
Step :  2  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  2  done.
Step :  3  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  3  done.
Step :  4  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Step :  4  done.
Step :  5  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  5  done.
Step :  6  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Step :  6  done.
Step :  7  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  7  done.
Step :  8  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  8  done.
Step :  9  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  9  done.
Step :  10  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Step :  10  done.
Step :  11  started...
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Step :  11  done.
Step :  12  started...
10/10 ━━━━━━━━━━━━━━━━━━━

In [77]:
import numpy as np
MeanMSE = np.mean(MeanSquareError)
STDMSE = np.std(MeanSquareError)

print("Mean for MSE: ",MeanMSE,"\nStandard Deviation for MSE: ",STDMSE)
#MeanMSE


Mean for MSE:  180.27650479920663 
Standard Deviation for MSE:  99.95014731187788
